<a href="https://colab.research.google.com/github/Egozit/ds_master_dataton/blob/main/02_team_hak_19_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Импорт

In [ ]:
import pandas as pd
import numpy as np
import requests 
import re

import json
from lxml import etree, objectify
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 

## Функции

In [ ]:
def html_table_read(html_table, to_erase=None):
    """
    This funtion transform BeautifulSoup tag object with table
    into list of columns. This version removes references from
    column names only.
    TODO: 1. Add references check
    TODO: 2. Refactoring is needed
    
    :param bs4.tag html_table:
    :param list to_erase: [optional] list of symbols which must 
                                     be removed from the results 
                                     (Default = [])
    
    :return: list (Content of the table), 
             list (Content of the column row)
    """
    
    if to_erase is None:
        to_erase = []    
    column_names = []
    table_content = []

    html_rows = html_table.find_all('tr')
    raw_column_names = html_rows[0].find_all('th')

    for element in raw_column_names:
        if element.find('sup') is not None:
            element.sup.decompose()
        name = re.sub("<.*?>", "", str(element))
        column_names.append(name)

    for row in html_rows:
        if row.td is not None:
            current_row = []
            row_elements = row.find_all('td')
            for element in range(0, len(row_elements)):
                value = re.sub("<.*?>", "", str(row_elements[element]))
                for symbol in to_erase:
                    value = value.replace(symbol, "")
                current_row.append(value)
            table_content.append(current_row)
    
    return(table_content, column_names)

def get_cov_area_tag_df(html_table):
    """
    This function parses data from html file into two-dimensional list.
    TODO: 1. Data cleaning
    TODO: 2. Reworking function describtion
    TODO: 3. Find a way to join it to main table
    """
    table = []
    for tag in html_table.find('div', style = "text-align: justify;").find_all('td'):
        table.append(tag.span.text)

    num_el = 7
    table = [table[i:i + num_el] for i in range(0, len(table), num_el)]   
    return table


### Загрузка таблицы районов города Москва

In [ ]:
target_page = ('https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0'
               + '%BA_%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BE%D0%B2_%D0%B8_%D0%BF%'
               + 'D0%BE%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B9_%D0%9C%D0%BE%'
               + 'D1%81%D0%BA%D0%B2%D1%8B')
to_drop = [' ', '\t', '\n', '\xa0']

districts_response = requests.get(target_page)

with open('msk_districts.html', 'w+b') as f:
    f.write(districts_response.content)

with open('msk_districts.html', encoding='UTF-8') as page:
    soup = BeautifulSoup(page, 'lxml')
    
dist_html_table = soup.find('table')
table_content, column_names = html_table_read(dist_html_table, to_drop)

raw_districts_df = pd.DataFrame(table_content, columns=column_names)
raw_districts_df.drop(columns=['№', 
                               'Флаг', 
                               'Герб', 
                               'Название района/поселения[3][4]', 
                               'Адми-нистра-тивныйокруг'], 
                      inplace=True, 
                      errors='ignore')
raw_districts_df.columns = ['dist_name', 
                            'dist_square', 
                            'dist_population', 
                            'dist_population_density', 
                            'living_space_m2', 
                            'living_space_m2_per_person']

In [ ]:
raw_districts_df.head()

,dist_name,dist_square,dist_population,dist_population_density,living_space_m2,living_space_m2_per_person
0,Академический,5.83,110038,18874.44,2467.0,22.7
1,Алексеевский,5.29,80634,15242.72,1607.9,20.5
2,Алтуфьевский,3.25,57697,17752.92,839.3,15.5
3,Арбат,2.11,36308,17207.58,731.0,26.0
4,Аэропорт,4.58,79541,17367.03,1939.7,25.9


In [ ]:
float(raw_districts_df.dist_square[0])

ValueError: ignored

# Данные по Зонам охвата магазинов

In [ ]:
main_domain_url = 'http://www.arhitrade.com/education.php?Id=43'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'}

coverage_area_response = requests.get(main_domain_url, headers=headers).text
soup = BeautifulSoup(coverage_area_response, 'html.parser')

raw_cov_area_df = get_cov_area_tag_df(soup)
cov_area_df = pd.DataFrame(raw_cov_area_df[1:], columns=raw_cov_area_df[0])

# Выгрузка 5 результатов в виде файла для демонстрации структуры датасета
cov_area_df.head(5)

,Тип магазина,Примерная площадь,Время доступа пешком,Время доступа на транспорте,Максимальное расстояние,Реальное расстояние,Радиус зоны охвата
0,Магазин шаговой доступности (мини-маркет),до 300 кв. м,до 15 минут,–,"~1,5 км",~1 км,"1,0 км"
1,"Универсальный магазин (миди-маркет, универма...",300–600 кв. м,до 30 минут,до 7 минут,"~3,7 км",~2 км,"2,5 км"
2,Средние специализированные и универсальные м...,600–2000 кв. м,до 60 минут,до 15 минут,"~7,7 км",~4 км,"5,0 км"
3,Крупные специализированные и универсальные м...,более 2000 кв. м,до 120 минут,до 30 минут,~15 км,~10 км,12 км


## Основные данные


In [ ]:
trade_place_data = pd.read_excel('https://github.com/Egozit/ds_master_dataton/raw/main/data/%D1%81%D1%82%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%B0%D1%80%D0%BD%D1%8B%D0%B5%20%D0%BE%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B.xlsx')
service_place_data = pd.read_excel('https://github.com/Egozit/ds_master_dataton/raw/main/data/%D0%91%D1%8B%D1%82%D0%BE%D0%B2%D1%8B%D0%B5%20%D1%83%D1%81%D0%BB%D1%83%D0%B3%D0%B8.xlsx')
tpu_data = pd.read_excel('https://github.com/Egozit/ds_master_dataton/raw/main/data/tpu.xlsx')

service_place_data.drop(columns=['Latitude_WGS84', 
                                 'Longitude_WGS84'], 
                        inplace=True, 
                        errors='ignore')

#Объединение данных торговых точек и точек услуг
trade_place_data.append(service_place_data)

#Создание 
trade_place_data['PlaceSize'] = 1
new_columns = ['tpu_'+str(x) for x in list(tpu_data.columns)]
trade_place_data[new_columns] = np.nan

# Выгрузка 5 результатов в виде файла для демонстрации структуры датасета
result = trade_place_data.head(5).to_json(orient="index", force_ascii=False).encode('utf8')
parsed = json.loads(result)
with open('middle_results.json', 'w', encoding='utf8') as json_file:
    json.dump(parsed, json_file, ensure_ascii=False, indent=4)

In [ ]:
# Выгрузка 5 результатов в виде файла для демонстрации структуры датасета
result = trade_place_data.head(5).to_json(orient="index", force_ascii=False).encode('utf8')
parsed = json.loads(result)
with open('middle_results.json', 'w', encoding='utf8') as json_file:
    json.dump(parsed, json_file, ensure_ascii=False, indent=4)

In [ ]:
trade_place_data.head()

,global_id,Name,IsNetObject,OperatingCompany,TypeService,TypeObject,AdmArea,District,Address,PublicPhone,WorkingHours,ClarificationOfWorkingHours,PlaceSize,tpu_TPUName,tpu_global_id,tpu_AdmArea,tpu_District,tpu_NearStation,tpu_YearOfComissioning,tpu_Status,tpu_AvailableTransfer,tpu_CarCapacity,latitude,longitude,tpu_latitude,tpu_longitude
0,637390942,Одежда. Обувь. Промтовары,нет,NaN,реализация непродовольственных товаров,Магазин «Промтовары»,Северный административный округ,район Беговой,"город Москва, Беговая аллея, дом 3А",(977) 757-40-37,День недели: понедельник\nЧасы работы: 09:30-2...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.564782,55.7833609999999,NaN,NaN
1,637391109,АЙКОВЕР,нет,NaN,реализация непродовольственных товаров,Магазин-салон,Центральный административный округ,Тверской район,"город Москва, Новослободская улица, дом 50/1, ...",(499) 250-12-40,День недели: понедельник\nЧасы работы: 10:00-2...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.59484948239575,55.78655886388782,NaN,NaN
2,637391684,Магазин «Радуга»,нет,NaN,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,район Марьино,"город Москва, Марьинский бульвар, дом 4",(909) 977-22-02,День недели: понедельник\nЧасы работы: 10:00-2...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.749818553954185,55.64795541517657,NaN,NaN
3,637391714,Точка любви,да,Точка любви,реализация непродовольственных товаров,Прочие специализированные непродовольственные ...,Северный административный округ,район Беговой,"город Москва, Ленинградский проспект, дом 12",(985) 780-51-78,День недели: понедельник\nЧасы работы: 10:00-2...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.578783498641165,55.78018806474115,NaN,NaN
4,637391844,ПРОДУКТЫ,нет,NaN,реализация продовольственных товаров,Магазин «Продукты»,Юго-Восточный административный округ,район Лефортово,"город Москва, Волочаевская улица, дом 40Б",(495) 361-17-35,День недели: понедельник\nЧасы работы: круглос...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.67813953250334,55.75248197832822,NaN,NaN


In [ ]:
trade_place_data.dtypes

global_id                        int64
Name                            object
IsNetObject                     object
OperatingCompany                object
TypeService                     object
TypeObject                      object
AdmArea                         object
District                        object
Address                         object
PublicPhone                     object
WorkingHours                    object
ClarificationOfWorkingHours     object
PlaceSize                        int64
tpu_TPUName                    float64
tpu_global_id                  float64
tpu_AdmArea                    float64
tpu_District                   float64
tpu_NearStation                float64
tpu_YearOfComissioning         float64
tpu_Status                     float64
tpu_AvailableTransfer          float64
tpu_CarCapacity                float64
latitude                        object
longitude                       object
tpu_latitude                   float64
tpu_longitude            

In [ ]:
trade_place_data['latitude'], trade_place_data['longitude'] = zip(*trade_place_data.geoData.str.strip('{type=Point, coordinates=[]}').str.split(', '))
trade_place_data = trade_place_data.drop(columns=['geoData'])
trade_place_data['tpu_latitude'] = np.nan
trade_place_data['tpu_longitude'] = np.nan
trade_place_data = trade_place_data.drop(columns=['tpu_geoData'])
trade_place_data = trade_place_data.drop(columns=['ID', 'tpu_ID'])

In [ ]:
tpu_data['tpu_latitude'], tpu_data['tpu_longitude'] = zip(*tpu_data.geoData.str.strip('{type=Point, coordinates=[]}').str.split(', '))
tpu_data = tpu_data.drop(columns=['geoData'])
